# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tosontsengel,48.7567,98.2839,274.07,48,40,1.38,MN,1746484299
1,1,qaqortoq,60.7167,-46.0333,272.33,90,85,0.54,GL,1746484300
2,2,wailua homesteads,22.0669,-159.3780,298.78,81,75,5.14,US,1746484302
3,3,ryotsu-minato,38.0833,138.4333,284.00,87,100,1.89,JP,1746484303
4,4,bikin,46.8161,134.2600,283.24,66,27,1.60,RU,1746484304


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    scale=1,
    color='City',
    cmap='Category20',
    frame_width=800,
    frame_height=500,
    hover_cols=['City', 'Humidity'],
    alpha=0.7
).opts(
    title='City Humidity'
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:

%%capture --no-display
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] >= 20) & (city_data_df["Humidity"] <= 25) & (city_data_df["Cloudiness"] < 60) & (city_data_df["Wind Speed"] < 65)]

# Drop any rows with null values
cleaned_df = ideal_weather_df.dropna()

# Display sample data
map_plot = cleaned_df.hvplot.points(
    'Lng', 'Lat',
    scale=1,
    color='City',
    frame_width=800,
    frame_height=500,
    geo=True,
    tiles='OSM'  # OpenStreetMap tiles to include state outlines
).opts(
    title='Ideal Weather Cities'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
print(hotel_df.head())

                City Country      Lat       Lng  Humidity Hotel Name
0       tosontsengel      MN  48.7567   98.2839        48           
1           qaqortoq      GL  60.7167  -46.0333        90           
2  wailua homesteads      US  22.0669 -159.3780        81           
3      ryotsu-minato      JP  38.0833  138.4333        87           
4              bikin      RU  46.8161  134.2600        66           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 3,  # Try to get up to 3 hotels to check availability
    "apiKey": geoapify_key
    # "apiKey": "384bdfc21de34ce2b3b3d3aa49f7d453"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
tosontsengel - nearest hotel: Алтай
qaqortoq - nearest hotel: Hotel Qaqortoq
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
ryotsu-minato - nearest hotel: ゆたかや旅館
bikin - nearest hotel: Эльдорадо
georgetown - nearest hotel: Page 63 hostel
west island - nearest hotel: Cocos Village Bungalows
'ain el melh - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
kingston - nearest hotel: Aphrodite Hotel
jaciara - nearest hotel: Pousada Rosa dos Ventos
port-aux-francais - nearest hotel: Keravel
ribeira grande - nearest hotel: Hospedaria JSF
fale old settlement - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
hithadhoo - nearest hotel: Pebbles Inn
nizhniy kuranakh - nearest hotel: Самородок
tura - nearest hotel: Hotel Polo Orchid
hermanus - nearest hotel: Aloe guest house
barwani - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solit

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tosontsengel,MN,48.7567,98.2839,48,Алтай
1,qaqortoq,GL,60.7167,-46.0333,90,Hotel Qaqortoq
2,wailua homesteads,US,22.0669,-159.3780,81,Hilton Garden Inn Kauai Wailua Bay
3,ryotsu-minato,JP,38.0833,138.4333,87,ゆたかや旅館
4,bikin,RU,46.8161,134.2600,66,Эльдорадо


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_plot=hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    hover_cols=["Hotel Name", "Country"],
    color="City",
    tiles="OSM"
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Hotel Name,Country)